In [1]:
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

pd.set_option("display.max_rows", 500)

from utilities import DataCleaner


def ranks(n):
    return list(range(1, n + 1))[::-1]


def remove_parentheses(string):
    return string.split("(")[0].strip()


def fix(df, idx, col, new):
    df.loc[idx, col] = new

# Scraping

This is a nice wrapper for getting the soup object from a url. 

## Pitchfork

In [2]:
pitchfork_url = (
    "https://pitchfork.com/features/lists-and-guides/the-200-best-albums-of-the-2010s/"
)

pitchfork = DataCleaner(pitchfork_url, "Pitchfork")

In [3]:
pitchfork.create_tags("h2")
pitchfork.split(": ")
pitchfork.transpose()
# pitchfork.albums = pitchfork.apply(pitchfork.albums, remove_parentheses)
pitchfork.extract_year_from_paren("albums")
pitchfork.apply("albums", remove_parentheses)
pitchfork.create_df(ranks(200))

# pitchfork.print_data()
pitchfork.df.head()

,artist,album,rank,genre,year,reviewer,reviewer_url
0,Ratking,So It Goes,200,None,2014,Pitchfork,https://pitchfork.com/features/lists-and-guide...
1,Wu Lyf,Go Tell Fire to the Mountain,199,None,2011,Pitchfork,https://pitchfork.com/features/lists-and-guide...
2,Jean Grae / Quelle Chris,Everything’s Fine,198,None,2018,Pitchfork,https://pitchfork.com/features/lists-and-guide...
3,Fatima Al Qadiri,Genre-Specific Xperience,197,None,2011,Pitchfork,https://pitchfork.com/features/lists-and-guide...
4,Portal,Vexovoid,196,None,2013,Pitchfork,https://pitchfork.com/features/lists-and-guide...


## billboard

In [4]:
bill_url = "https://www.billboard.com/articles/news/list/8543722/best-albums-of-the-2010s-top-100"
bill = DataCleaner(bill_url, "Billboard")
bill.create_tags("strong")
bill.strings.insert(25, "75. Charli XCX, True Romance (2013)")
bill.strings.insert(41, "59. Pusha T, Daytona (2018)")
bill.strings.insert(50, "50. Carly Rae Jepsen, E•MO•TION (2015)")
bill.trim("strings", chars="1234567890. ")
bill.extract_year_from_paren("strings")
bill.split(",")
bill.transpose()
bill.apply("albums", remove_parentheses)
bill.apply("albums", lambda x: x.replace("\xa0", " "))
bill.trim("albums", chars="\xa0", right=True)
bill.create_df(ranks(100))
fix(bill.df, bill.df["artist"].str.startswith("Tyler"), "artist", "Tyler, the Creator")
fix(bill.df, bill.df["artist"].str.startswith("Tyler"), "album", "IGOR")

# bill.print_data()
bill.df.head()

,artist,album,rank,genre,year,reviewer,reviewer_url
0,Lady Gaga & Bradley Cooper,A Star Is Born Soundtrack,100,None,2018,Billboard,https://www.billboard.com/articles/news/list/8...
1,Lady Antebellum,Need You Now,99,None,2010,Billboard,https://www.billboard.com/articles/news/list/8...
2,Japandroids,Celebration Rock,98,None,2012,Billboard,https://www.billboard.com/articles/news/list/8...
3,Porter Robinson,Worlds,97,None,2014,Billboard,https://www.billboard.com/articles/news/list/8...
4,Ed Sheeran,x,96,None,2014,Billboard,https://www.billboard.com/articles/news/list/8...


In [5]:
len(bill)

100

## Stereogum

In [31]:
stereo_url = "https://www.stereogum.com/featured/best-albums-of-the-2010s-list/"
stereo = DataCleaner(stereo_url, "Stereogum")
stereo.create_tags("h2")
stereo.strings = stereo.strings[1::2]
stereo.trim("strings", "1234567890 ")
stereo.split(" – ")
stereo.transpose()
stereo.extract_year_from_paren("albums", ",)")
stereo.apply("albums", remove_parentheses)
stereo.create_df(ranks(100))
fix(stereo.df, stereo.df["album"].str.startswith("Rich Gang"), "artist", "Rich Gang")

# stereo.print_data()
stereo.df.head()

,artist,album,rank,genre,year,reviewer,reviewer_url
0,Future,Pluto,100,None,2012,Stereogum,https://www.stereogum.com/featured/best-albums...
1,Car Seat Headrest,Teens Of Denial,99,None,2016,Stereogum,https://www.stereogum.com/featured/best-albums...
2,PUP,The Dream Is Over,98,None,2016,Stereogum,https://www.stereogum.com/featured/best-albums...
3,Courtney Barnett,"Sometimes I Sit And Think, And Sometimes I Jus...",97,None,2015,Stereogum,https://www.stereogum.com/featured/best-albums...
4,Colleen Green,I Want To Grow Up,96,None,2015,Stereogum,https://www.stereogum.com/featured/best-albums...


## Consequence of Sound 

In [47]:
cos_url =  "https://consequenceofsound.net/2019/11/top-albums-of-the-2010s/full-post/"
cos = DataCleaner(cos_url, "Consequence_of_Sound")
cos.create_tags("")

# cos.strings

In [128]:
cos_rank = []
artist = []
album = []
year = []

for tag in soup.find_all('h2')[2:-1]:
    cos_rank.append(tag.contents[0].split('.')[0])
    artist.append(tag.contents[0].split('.', 1)[1][1:-3])
    album.append(tag.contents[1].contents[0])
    year.append(tag.contents[2][2:-1])

In [129]:
cos_df = combine_to_df(
    album_list=album,
    artist_list=artist,
    rank_list=cos_rank,
    year_list=year,
    url=url,
)

# fixes
cos_df.iloc[67, 3] = '2015'
cos_df.iloc[83, 3] = '2016'
cos_df.iloc[89, 3] = '2012'

In [130]:
cos_df.head()

,artist,album,rank,year,reviewer_url
0,PJ Harvey,Let England Shake,100,2011,https://consequenceofsound.net/2019/11/top-alb...
1,Savages,Silence Yourself,99,2013,https://consequenceofsound.net/2019/11/top-alb...
2,Destroyer,Kaputt,98,2011,https://consequenceofsound.net/2019/11/top-alb...
3,Ariana Grande,"thank you, next",97,2019,https://consequenceofsound.net/2019/11/top-alb...
4,Bon Iver,"22, A Million",96,2016,https://consequenceofsound.net/2019/11/top-alb...


## Time

In [60]:
time_url =  "https://time.com/5725768/best-albums-2010s-decade/"
time = DataCleaner(time_url, "Time")
time.create_tags("strong")
time.strings = time.strings[:-1]
time.split(", ")
time.transpose()
time.extract_year_from_paren("albums")
time.apply("albums", remove_parentheses)
time.create_df(10)

# time.print_data()
time.df

,artist,album,rank,genre,year,reviewer,reviewer_url
0,Fiona Apple,The Idler Wheel…,10,None,2012,Time,https://time.com/5725768/best-albums-2010s-dec...
1,Miguel,Kaleidoscope Dream,10,None,2012,Time,https://time.com/5725768/best-albums-2010s-dec...
2,Beauty Pill,Beauty Pill Describes Things As They Are,10,None,2015,Time,https://time.com/5725768/best-albums-2010s-dec...
3,Carly Rae Jepsen,E•MO•TION,10,None,2015,Time,https://time.com/5725768/best-albums-2010s-dec...
4,Beyoncé,Lemonade,10,None,2016,Time,https://time.com/5725768/best-albums-2010s-dec...
5,Leonard Cohen,You Want It Darker,10,None,2016,Time,https://time.com/5725768/best-albums-2010s-dec...
6,Miranda Lambert,The Weight of These Wings,10,None,2016,Time,https://time.com/5725768/best-albums-2010s-dec...
7,Solange,A Seat At the Table,10,None,2016,Time,https://time.com/5725768/best-albums-2010s-dec...
8,Kendrick Lamar,DAMN.,10,None,2017,Time,https://time.com/5725768/best-albums-2010s-dec...
9,Ozuna,Aura,10,None,2018,Time,https://time.com/5725768/best-albums-2010s-dec...


## Paste Magazine

In [61]:
paste_urls = [
    "https://www.pastemagazine.com/articles/2019/10/best-albums-of-the-2010s-paste.html",
    "https://www.pastemagazine.com/articles/2019/10/best-albums-of-the-2010s-paste.html?p=2",
    "https://www.pastemagazine.com/articles/2019/10/best-albums-of-the-2010s-paste.html?p=3",
    "https://www.pastemagazine.com/articles/2019/10/best-albums-of-the-2010s-paste.html?p=4",
]

paste_1, paste_2, paste_3, paste_4 = [
    DataCleaner(url, "Paste" + str(paste_urls.index(url))) for url in paste_urls
]

In [77]:
paste_1.create_tags("b")
paste_1.strings = paste_1.strings[2:26]
paste_1.trim("strings", "1234567890. ")
paste_1.split(": ")
paste_1.transpose()
paste_1.extract_year_from_paren("albums")
paste_1.apply("albums", remove_parentheses)
paste_1.create_df(ranks(100)[:24])

# paste_1.print_data()
paste_1.df

,artist,album,rank,genre,year,reviewer,reviewer_url
0,Drake,Take Care,100,None,2011,Paste0,https://www.pastemagazine.com/articles/2019/10...
1,Japanese Breakfast,Soft Sounds from Another Planet,99,None,2017,Paste0,https://www.pastemagazine.com/articles/2019/10...
2,Nick Cave & The Bad Seeds,Skeleton Tree,98,None,2016,Paste0,https://www.pastemagazine.com/articles/2019/10...
3,Deafheaven,Sunbather,97,None,2013,Paste0,https://www.pastemagazine.com/articles/2019/10...
4,"Tyler, The Creator",Flower Boy,96,None,2017,Paste0,https://www.pastemagazine.com/articles/2019/10...
5,The Black Keys,Brothers,95,None,2010,Paste0,https://www.pastemagazine.com/articles/2019/10...
6,Sky Ferreira,"Night Time, My Time",94,None,2013,Paste0,https://www.pastemagazine.com/articles/2019/10...
7,Charles Bradley,Victim of Love,93,None,2013,Paste0,https://www.pastemagazine.com/articles/2019/10...
8,Travis Scott,ASTROWORLD,92,None,2018,Paste0,https://www.pastemagazine.com/articles/2019/10...
9,Lucius,Wildewoman,91,None,2013,Paste0,https://www.pastemagazine.com/articles/2019/10...


In [82]:
paste_2.create_tags("b", class_="big")

paste_2.trim("strings", "1234567890. ")
paste_2.split(": ")
paste_2.transpose()
paste_2.extract_year_from_paren("albums")
paste_2.apply("albums", remove_parentheses)
paste_2.create_df(ranks(100)[24:51])

# paste_2.print_data()
paste_2.df

,artist,album,rank,genre,year,reviewer,reviewer_url
0,IDLES,Joy As An Act of Resistance,76,None,2018,Paste1,https://www.pastemagazine.com/articles/2019/10...
1,Fiona Apple,The Idler Wheel Is Wiser than the Driver of th...,75,None,2012,Paste1,https://www.pastemagazine.com/articles/2019/10...
2,Sturgill Simpson,Metamodern Sounds in Country Music,74,None,2014,Paste1,https://www.pastemagazine.com/articles/2019/10...
3,Kevin Morby,City Music,73,None,2017,Paste1,https://www.pastemagazine.com/articles/2019/10...
4,Frightened Rabbit,The Winter of Mixed Drinks,72,None,2010,Paste1,https://www.pastemagazine.com/articles/2019/10...
5,Caribou,Our Love,71,None,2014,Paste1,https://www.pastemagazine.com/articles/2019/10...
6,Destroyer,Kaputt,70,None,2011,Paste1,https://www.pastemagazine.com/articles/2019/10...
7,Leon Bridges,Coming Home,69,None,2015,Paste1,https://www.pastemagazine.com/articles/2019/10...
8,The Decemberists,The King is Dead,68,None,2011,Paste1,https://www.pastemagazine.com/articles/2019/10...
9,Cardi B,Invasion of Privacy,67,None,2018,Paste1,https://www.pastemagazine.com/articles/2019/10...


In [88]:
paste_3.create_tags("b", class_="big")

paste_3.trim("strings", "1234567890. ")
paste_3.split(": ")
paste_3.transpose()
paste_3.extract_year_from_paren("albums")
paste_3.apply("albums", remove_parentheses)
paste_3.create_df(ranks(100)[51:75])

# paste_3.print_data()
paste_3.df

,artist,album,rank,genre,year,reviewer,reviewer_url
0,A Tribe Called Quest,We got it from Here… Thank You 4 Your service,49,None,2016,Paste2,https://www.pastemagazine.com/articles/2019/10...
1,Alabama Shakes,Boys & Girls,48,None,2012,Paste2,https://www.pastemagazine.com/articles/2019/10...
2,Vince Staples,Summertime ‘06,47,None,2015,Paste2,https://www.pastemagazine.com/articles/2019/10...
3,Chance the Rapper,Coloring Book,46,None,2016,Paste2,https://www.pastemagazine.com/articles/2019/10...
4,Snail Mail,Lush,45,None,2018,Paste2,https://www.pastemagazine.com/articles/2019/10...
5,Tame Impala,Lonerism,44,None,2012,Paste2,https://www.pastemagazine.com/articles/2019/10...
6,Weyes Blood,Titanic Rising,43,None,2019,Paste2,https://www.pastemagazine.com/articles/2019/10...
7,Janelle Monáe,Dirty Computer,42,None,2018,Paste2,https://www.pastemagazine.com/articles/2019/10...
8,Japandroids,Celebration Rock,41,None,2012,Paste2,https://www.pastemagazine.com/articles/2019/10...
9,Beyoncé,Beyoncé,40,None,2013,Paste2,https://www.pastemagazine.com/articles/2019/10...


In [92]:
paste_4.create_tags("b", class_="big")

paste_4.trim("strings", "1234567890. ")
paste_4.split(": ")
paste_4.transpose()
paste_4.extract_year_from_paren("albums")
paste_4.apply("albums", remove_parentheses)
paste_4.create_df(ranks(100)[75:])

# paste_4.print_data()
paste_4.df

,artist,album,rank,genre,year,reviewer,reviewer_url
0,Kanye West,Yeezus,25,None,2013,Paste3,https://www.pastemagazine.com/articles/2019/10...
1,The War on Drugs,Lost in the Dream,24,None,2014,Paste3,https://www.pastemagazine.com/articles/2019/10...
2,Mitski,Puberty 2,23,None,2016,Paste3,https://www.pastemagazine.com/articles/2019/10...
3,Father John Misty,I Love You Honeybear,22,None,2015,Paste3,https://www.pastemagazine.com/articles/2019/10...
4,Arcade Fire,The Suburbs,21,None,2010,Paste3,https://www.pastemagazine.com/articles/2019/10...
5,Lucy Dacus,Historian,20,None,2018,Paste3,https://www.pastemagazine.com/articles/2019/10...
6,Alvvays,Antisocialites,19,None,2017,Paste3,https://www.pastemagazine.com/articles/2019/10...
7,Robyn,Body Talk,18,None,2010,Paste3,https://www.pastemagazine.com/articles/2019/10...
8,Radiohead,A Moon Shaped Pool,17,None,2016,Paste3,https://www.pastemagazine.com/articles/2019/10...
9,Beach House,Teen Dream,16,None,2010,Paste3,https://www.pastemagazine.com/articles/2019/10...


## Genius

In [104]:
genius_url = "https://genius.com/a/the-genius-communitys-100-best-albums-of-the-2010s"
genius = DataCleaner(genius_url, "Genius")

genius.create_tags("div", class_="g_list-item-header")
genius.trim("strings", "\n 1234567890")
genius.trim("strings", "\n ", left=False, right=True)
genius.split(" – ")
genius.transpose()
genius.create_df(ranks(100))

# genius.print_data()
genius.df.head()

,artist,album,rank,genre,reviewer,reviewer_url
0,Justin Bieber,Purpose,100,None,Genius,https://genius.com/a/the-genius-communitys-100...
1,Katy Perry,Teenage Dream,99,None,Genius,https://genius.com/a/the-genius-communitys-100...
2,Bon Iver,"Bon Iver, Bon Iver",98,None,Genius,https://genius.com/a/the-genius-communitys-100...
3,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",97,None,Genius,https://genius.com/a/the-genius-communitys-100...
4,Schoolboy Q,Oxymoron,96,None,Genius,https://genius.com/a/the-genius-communitys-100...


## Rolling Stone

In [105]:
roll_urls = [
  "https://www.rollingstone.com/music/music-lists/best-albums-2010s-ranked-913997/",
    "https://www.rollingstone.com/music/music-lists/best-albums-2010s-ranked-913997/carly-rae-jepsen-emotion-album-917470/"
] 
roll_1, roll_2 = [DataCleaner(url, "Rolling_Stone" + str(roll_urls.index(url))) for url in roll_urls]

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [143]:
rs_rank = []
album = []
artist = []

for soup in soup_list:
    for tag in soup.find_all("header", class_="c-list__header"):
        rs_rank.append(tag.span.text.strip())
        artist.append(tag.h3.text.strip().split(", ", 1)[0])
        if len(tag.h3.text.strip().split(", ", 1)) == 1:
            album.append('Hamilton')
        else:
            album.append(tag.h3.text.strip().split(", ", 1)[1][1:-1])

In [144]:
rs_df = combine_to_df(
    album_list=album,
    artist_list=artist,
    rank_list=genius_rank,
    url=urls[0],
)

In [145]:
rs_df.head()

,artist,album,rank,reviewer_url
0,The War on Drugs,A Deeper Understanding,100,https://www.rollingstone.com/music/music-lists...
1,Lana Del Rey,Ultraviolence,99,https://www.rollingstone.com/music/music-lists...
2,John Prine,The Tree of Forgiveness,98,https://www.rollingstone.com/music/music-lists...
3,Rich Gang,Tha Tour Part 1,97,https://www.rollingstone.com/music/music-lists...
4,Low Cut Connie,Call Me Sylvia,96,https://www.rollingstone.com/music/music-lists...


# Combining Data 

## Reformatting

In [146]:
def reformat(df_list):
    """
    Trims any external whitespaces on all strings and turns rank and year into integers.
    """
    for df in df_list:

        # trim any whitespace around the strings in each column
        for column in df.columns:
            if df[column].dtype == 'object':
                df[column] = df[column].apply(lambda x: x.strip())

        # Make years and ranks integers
        
#         if "year" in df.columns:
#             df["year"] = df["year"].astype('Int32')
            
        df["rank"] = df["rank"].astype(int)

    return

In [147]:
df_list = [
    pitchfork_df,
    billboard_df,
    stereogum_df,
    cos_df,
    time_df,
    paste_df,
    genius_df,
    rs_df
]

df_urls = [df['reviewer_url'].iloc[0] for df in df_list]

url_names = [
    "Pitchfork",
    "Billboard",
    "Stereogum",
    "Consequence of Sound",
    "Time",
    "Paste",
    "Genius",
    "Rolling Stone"
]

reviewer_dict = dict(zip(df_urls, url_names))

reformat(df_list)

In [148]:
albums = pd.concat(df_list, sort=False)

In [149]:
# albums['year'] = albums['year'].astype('Int64')
albums.reset_index()

,index,artist,album,rank,year,reviewer_url
0,0,Ratking,So It Goes,200,2014,https://pitchfork.com/features/lists-and-guide...
1,1,Wu Lyf,Go Tell Fire to the Mountain,199,2011,https://pitchfork.com/features/lists-and-guide...
2,2,Jean Grae / Quelle Chris,Everything’s Fine,198,2018,https://pitchfork.com/features/lists-and-guide...
3,3,Fatima Al Qadiri,Genre-Specific Xperience,197,2011,https://pitchfork.com/features/lists-and-guide...
4,4,Portal,Vexovoid,196,2013,https://pitchfork.com/features/lists-and-guide...
...,...,...,...,...,...,...
805,95,David Bowie,Blackstar,5,NaN,https://www.rollingstone.com/music/music-lists...
806,96,Taylor Swift,Red,4,NaN,https://www.rollingstone.com/music/music-lists...
807,97,Kendrick Lamar,To Pimp a Butterfly,3,NaN,https://www.rollingstone.com/music/music-lists...
808,98,Beyoncé,Lemonade,2,NaN,https://www.rollingstone.com/music/music-lists...


In [150]:
artist_list = albums.artist.unique()

In [151]:
len(artist_list)

332

## Checking lengths

In [152]:
for df in df_list:
    print(len(df))

200
100
100
100
10
100
100
100


## Fixing Album Names

Some albums appear with a different name. For example, David Bowie's _Blackstar_.

In [153]:
def get_albums_from_artist(artist, df=albums):
    return list(set(df.loc[df.artist == artist, "album"].to_list()))


def create_fuzz_matrix(album_list):
    """
    Creates a confusion matrix using fuzzy finding.
    """
    fuzz_mat = np.zeros((len(album_list), len(album_list)), dtype=np.int)

    for i in range(len(album_list)):
        for j in range(i + 1, len(album_list)):
            fuzz_mat[i, j] = fuzz.partial_ratio(album_list[i], album_list[j])
            fuzz_mat[j, i] = fuzz_mat[i, j]
        fuzz_mat[i, i] = 100

    return fuzz_mat


def get_oddly_formatted_albums(albums_df, artist_list=artist_list, similarity=80):
    oddly_formatted_artists = []

    for artist in artist_list:
        album_list = get_albums_from_artist(artist, df=albums_df)
        if len(album_list) == 1:
            # no need to make changes since the album appears once.
            continue
        else:
            # get upper diagonal
            fuzz_mat = np.triu(create_fuzz_matrix(album_list), 1)

            # check if there are very similar things on the off-diagonal.
            # this will indicate the same album appears with slightly
            # different spelling.
            if fuzz_mat[fuzz_mat > similarity].size > 0:
                oddly_formatted_artists.append(artist)

    return oddly_formatted_artists

In [154]:
for artist in get_oddly_formatted_albums(albums):
    print(get_albums_from_artist(artist))

['I Like It When You Sleep, For You Are So Beautiful Yet So Unaware of It', 'I like it when you sleep…', 'I Like It When You Sleep, For You Are So Beautiful Yet So Unaware Of It', 'A Brief Inquiry Into Online Relationships']
['Teens Of Denial', 'Teens of Denial', 'Twin Fantasy (Face to Face)']
['Thank U, Next', 'Sweetener', 'thank you, next', 'Thank U Next', 'Dangerous Woman']
['Modern Vampires of the City', 'Father Of The Bride', 'Modern Vampires Of The City', 'Contra', 'Father of the Bride']
['Bon Iver', '22, A Million', 'Bon Iver, Bon Iver']
['Sremmlife', 'SremmLife']
['When I Get Home', 'A Seat At the Table', 'A Seat at the Table', 'A Seat At The Table']
['Smoke Ring for My Halo', 'Smoke Ring For My Halo']
['Invasion Of Privacy', 'Invasion of Privacy']
['I Love You Honeybear', 'I Love You, Honeybear']
['If You’re Reading This, It’s Too Late', 'If You’re Reading This It’s Too Late', 'Take Care', 'Nothing Was The Same', 'Nothing Was the Same']
['Sometimes I Sit and Think and Sometime

In [155]:
# Ultimately, the simplest thing for these was to write them in by
# hand. I would love ideas for another solution.
oddly_formatted_artists = get_oddly_formatted_albums(albums)
oddly_formatted_artists.append("Billie Eilish")
oddly_formatted_artists.append("Daughters")
oddly_formatted_artists.append("Carly Rae Jepsen")

oddly_formatted_albums = [
    ["I Like it when you sleep...", "A Brief Inquiry Into Online Relationships"],
    ["Teens of Denial", "Twin Fantasy (Face to Face)"],
    ["thank u, next", "Sweetener", "Dangerous Woman"],
    ["Father of the Bride", "Contra", "Modern Vampires of the City"],
    ["22, A Million", "Bon Iver"],
    ["SremmLife"],
    ["When I Get Home", "A Seat at the Table"],
    ["Smoke Ring for My Halo"],
    ["Invasion of Privacy"],
    ["I Love You, Honeybear"],
    ["Nothing Was the Same", "If You're Reading This, It's Too Late", "Take Care"],
    ["Sometimes I Sit and Think, and Sometimes I Just Sit"],
    ["good kid, m.A.A.d city", "DAMN.", "To Pimp a Butterfly"],
    ["Shaking the Habitual"],
    ["A Crow Looked at Me"],
    ["We Got It From Here... Thank You 4 Your Service"],
    ["Blackstar"],
    ["El Mal Querer"],
    ["Summertime '06", "Big Fish Theory"],
    ["Burn Your Fire for No Witness", "My Woman", "All Mirrors"],
    ["Norman Fucking Rockwell!", "Ultraviolence", "Born to Die"],
    ["Have One On Me"],
    ["Channel ORANGE", "Blonde", "Nostalgia Ultra"],
    ["The Idler Wheel..."],
    ["By the Way, I Forgive You"],
    ["Soft Sounds from Another Planet"],
    ["Rich Gang: Tha Tour Pt. 1"],
    ["When We All Fall Asleep, Where Do We Go?"],
    ["You Won't Get What You Want"],
    ["E•MO•TION"]    
]

odd_format_dict = dict(zip(oddly_formatted_artists, oddly_formatted_albums))

In [156]:
# Fix the album titles 

# For the one David Bowie album
albums.loc[albums['album'] == '★', "album"] = "Blackstar"

# Use fuzzyfinding to extract one album name for the weirdly formatted album titles.
albums['new_album_name'] = albums.apply(
    lambda row: process.extract(row['album'], odd_format_dict[row['artist']], limit=1)[0][0] if row['artist'] in oddly_formatted_artists else row['album'],
    axis=1
)

## Fixing missing years

Some albums have an incorrectly listed year, while others might have a missing year altogether. The following blocks aims to search the rest of the dataframe for the most common year.

In [157]:
def most_common(lst):
    """ Taken from 
    https://stackoverflow.com/questions/1518522/find-the-most-common-element-in-a-list
    """
    if lst:
        return max(set(lst), key=lst.count)
    else:
        return np.nan
    
def get_year(album, album_df=albums):
    
    years = list(album_df.loc[album_df["new_album_name"] == album, "year"].dropna().unique())
    
    if len(years) == 1:
        return years[0]
    elif len(years) > 1:
        return most_common(years)
    else:
        return np.nan


In [158]:
albums['new_year'] = albums['year']

albums.loc[albums['year'].isna(), 'new_year'] = albums.loc[albums['year'].isna(), "new_album_name"].apply(get_year)

Fill the remaining by hand.

In [159]:
albums['new_year'].isna().sum()

65

In [160]:
albums.loc[albums['new_year'].isna(), :]

,artist,album,rank,year,reviewer_url,new_album_name,new_year
4,Schoolboy Q,Oxymoron,96,NaN,https://genius.com/a/the-genius-communitys-100...,Oxymoron,NaN
5,Ariana Grande,Dangerous Woman,95,NaN,https://genius.com/a/the-genius-communitys-100...,Dangerous Woman,NaN
6,Isaiah Rashad,Cilvia Demo,94,NaN,https://genius.com/a/the-genius-communitys-100...,Cilvia Demo,NaN
7,alt-j,An Awesome Wave,93,NaN,https://genius.com/a/the-genius-communitys-100...,An Awesome Wave,NaN
9,Logic,Under Pressure,91,NaN,https://genius.com/a/the-genius-communitys-100...,Under Pressure,NaN
11,Kali Uchis,Isolation,89,NaN,https://genius.com/a/the-genius-communitys-100...,Isolation,NaN
12,Big Sean,Dark Sky Paradise,88,NaN,https://genius.com/a/the-genius-communitys-100...,Dark Sky Paradise,NaN
16,St. Vincent,MASSEDUCTION,84,NaN,https://genius.com/a/the-genius-communitys-100...,MASSEDUCTION,NaN
17,Daughters,You Won’t Get What You Want,83,NaN,https://genius.com/a/the-genius-communitys-100...,You Won't Get What You Want,NaN
18,Migos,Culture,82,NaN,https://genius.com/a/the-genius-communitys-100...,Culture,NaN


In [161]:
custom_year_dict = {
    "Oxymoron" : '2014',
    'Dangerous Woman' : '2016',
    'Cilvia Demo' : '2014',
    'An Awesome Wave' : '2012',
    'Under Pressure' : '2014',
    'Isolation' : '2018',
    'Dark Sky Paradise' : '2015',
    'MASSEDUCTION' : '2017',
    "You Won't Get What You Want" : '2018',
    'Culture' : '2017',
    'You Never Walk Alone' : '2017',
    'Twin Fantasy (Face to Face)' : '2018',
    'Stoney' : '2016',
    'Tetsuo & Youth' : '2015',
    'MADE' : '2016',
    'KIDS SEE GHOSTS' : '2018',
    'Night Visions' : '2012',
    'Blank Face LP' : '2016',
    'Konnichiwa' : '2016',
    'Rich Gang: Tha Tour Pt. 1' : '2014',
    'SATURATION III' : '2017',
    'Watching Movies With The Sound Off' : '2013',
    'Die Lit' : '2018',
    'Man On the Moon II: The Legend of Mr. Rager' : '2010',
    'TA1300' : '2018',
    'Teflon Don' : '2010',
    '1999' : '2012',
    'Rodeo' : '2015',
    'Piñata' : '2014',
    'My Krazy Life' : '2014',
    'Doo-Wops & Hooligans' : '2010',
    'Atrocity Exhibition' : '2016',
    'The Black Messiah' : '2014',
    'Because The Internet' : '2013',
    'DAYTONA' : '2018',
    'Watch The Throne' : '2011',
    'ANTi' : '2016',
    'A Deeper Understanding' : '2017',
    'The Tree of Forgiveness' : '2018',
    'Call Me Sylvia' : '2012',
    'Monster' : '2014',
    'Need to Feel Your Love' : '2017',
    'Songs of Innocence' : '2014',
    'Psychedelic Pill' : '2012',
    'Hardwired … to Self-Destruct' : '2016',
    'In Color' : '2015',
    'Chief' : '2011',
    'Mirror Traffic' : '2011',
    'The Highwomen' : '2019',
    'Nostalgia Ultra' : '2011',
    'On the Line' : '2019',
    'Blunderbuss' : '2012',
    'X 100Pre' : '2018',
    'Egypt Station' : '2018',
    'Harry Styles' : '2017',
    'American Teen' : '2017',
    'Wrecking Ball' : '2012',
    'Hamilton' : '2015',
    'Tempest' : '2012', 
    'Platinum' : '2014',
    "A Sailor’s Guide to Earth" : '2016',
    'Dark Matter' : '2017',
    'So Beautiful or So What' : '2011',
    'Interstate Gospel' : '2018'
}

In [162]:
albums.loc[albums['new_year'].isna(), "new_year"] = albums.loc[albums['new_year'].isna(), "new_album_name"].apply(lambda x: custom_year_dict[x])

In [163]:
albums['new_year'].isna().sum()

0

## Checking Artists

To check that artist names don't appear in multiple places with slightly different strings, the fuzz matrix will give us the similarity scores for all pairs of artist strings. Scores closer to 100 mean very similar, while close to 0 means dissimilar. Large off-diagonal terms mean two distinct strings that are highly similar.

In [164]:
np.sum(np.triu(create_fuzz_matrix(artist_list), 1) > 80)

44

Looks like 44 artists with similar names.

In [165]:
np.sum(np.triu(create_fuzz_matrix(artist_list), 1) > 99)

19

These 19 are highly similar. 

In [166]:
i_list, j_list = np.where(np.triu(create_fuzz_matrix(artist_list), 1) == 100)
i_list = list(i_list)
j_list = list(j_list)

In [167]:
duplicate_artists = list(set([artist_list[i] for i in i_list + j_list]))

In [168]:
duplicate_artists

['Future',
 'Future Islands',
 'Young Thug',
 'Arcade Fire',
 'Lady Gaga',
 'JAY-Z / Kanye West',
 'Jay-Z and Kanye West',
 'Kanye West',
 'U.S. Girls',
 'Arca',
 'Young Thug, Birdman, & Rich Homie Quan',
 'D’Angelo',
 'D’Angelo & the Vanguard',
 "D'Angelo and the Vanguard",
 'Girls',
 'JAY-Z & Kanye West',
 'Lady Gaga & Bradley Cooper',
 'Jay-Z & Kanye West',
 'Jay-Z',
 'Kanye and Jay-Z',
 "D'Angelo",
 'JAY-Z',
 'D’Angelo and the Vanguard',
 'Low Cut Connie',
 'D’Angelo And The Vanguard',
 'Low']

In [169]:
duplicate_dict = {
    "JAY-Z": ["Jay-Z", "JAY-Z", "Jay-z", "JAY-z"],
    "D'Angelo and the Vanguard": [
        "D’Angelo And The Vanguard",
        "D'Angelo and the Vanguard",
        "D’Angelo and the Vanguard",
        "D’Angelo & the Vanguard",
        "D'Angelo",
        'D’Angelo',
    ],
    "JAY-Z & Kanye West": [
        "JAY-Z & Kanye West",
        "Jay-Z & Kanye West",
        "Jay-Z and Kanye West",
        "JAY-Z / Kanye West",
    ],
}

In [170]:
# Make all duplicates the same. 
for artist in duplicate_dict.keys():
    albums.loc[albums["artist"].isin(duplicate_dict[artist]), "artist"] = artist

## Fixing "Hamilton"

The soundtrack to "Hamilton" appeared a few times with different attributions to the artist and different stylings.

In [171]:
hamiltons = albums.album.apply(lambda x: "Hamilton" in x) 
albums.loc[hamiltons, :]

,artist,album,rank,year,reviewer_url,new_album_name,new_year
53,Various Artists,Hamilton Original Cast Album,44,2015,https://www.billboard.com/articles/news/list/8...,Hamilton Original Cast Album,2015
73,Lin-Manuel Miranda,Hamilton: Original Broadway Cast Recording,27,2015,https://www.pastemagazine.com/articles/2019/10...,Hamilton: Original Broadway Cast Recording,2015
55,‘Hamilton’ Original Broadway Cast Recording,Hamilton,45,NaN,https://www.rollingstone.com/music/music-lists...,Hamilton,2015


In [172]:
albums.loc[hamiltons, "artist"] = "'Hamilton' Original Broadway Cast Recording"
albums.loc[hamiltons, "new_album_name"] = "Hamilton"
albums.loc[hamiltons, "new_year"] = '2015'

## Get reviewer

In [173]:
albums["reviewer"] = albums.reviewer_url.apply(lambda x: reviewer_dict[x])

## Final Cleanup

In [174]:
albums.head()

,artist,album,rank,year,reviewer_url,new_album_name,new_year,reviewer
0,Ratking,So It Goes,200,2014,https://pitchfork.com/features/lists-and-guide...,So It Goes,2014,Pitchfork
1,Wu Lyf,Go Tell Fire to the Mountain,199,2011,https://pitchfork.com/features/lists-and-guide...,Go Tell Fire to the Mountain,2011,Pitchfork
2,Jean Grae / Quelle Chris,Everything’s Fine,198,2018,https://pitchfork.com/features/lists-and-guide...,Everything’s Fine,2018,Pitchfork
3,Fatima Al Qadiri,Genre-Specific Xperience,197,2011,https://pitchfork.com/features/lists-and-guide...,Genre-Specific Xperience,2011,Pitchfork
4,Portal,Vexovoid,196,2013,https://pitchfork.com/features/lists-and-guide...,Vexovoid,2013,Pitchfork


In [175]:
albums['year'] = albums['new_year']
albums['album'] = albums['new_album_name']

albums.drop(['new_year', 'new_album_name'], axis=1, inplace=True)

In [176]:
albums.reset_index(drop=True)

,artist,album,rank,year,reviewer_url,reviewer
0,Ratking,So It Goes,200,2014,https://pitchfork.com/features/lists-and-guide...,Pitchfork
1,Wu Lyf,Go Tell Fire to the Mountain,199,2011,https://pitchfork.com/features/lists-and-guide...,Pitchfork
2,Jean Grae / Quelle Chris,Everything’s Fine,198,2018,https://pitchfork.com/features/lists-and-guide...,Pitchfork
3,Fatima Al Qadiri,Genre-Specific Xperience,197,2011,https://pitchfork.com/features/lists-and-guide...,Pitchfork
4,Portal,Vexovoid,196,2013,https://pitchfork.com/features/lists-and-guide...,Pitchfork
...,...,...,...,...,...,...
805,David Bowie,Blackstar,5,2016,https://www.rollingstone.com/music/music-lists...,Rolling Stone
806,Taylor Swift,Red,4,2012,https://www.rollingstone.com/music/music-lists...,Rolling Stone
807,Kendrick Lamar,To Pimp a Butterfly,3,2015,https://www.rollingstone.com/music/music-lists...,Rolling Stone
808,Beyoncé,Lemonade,2,2016,https://www.rollingstone.com/music/music-lists...,Rolling Stone


In [177]:
albums['year'] = albums['year'].astype(int)
albums['rank'] = albums['rank'].astype(int)

In [178]:
albums.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 810 entries, 0 to 99
Data columns (total 6 columns):
artist          810 non-null object
album           810 non-null object
rank            810 non-null int64
year            810 non-null int64
reviewer_url    810 non-null object
reviewer        810 non-null object
dtypes: int64(2), object(4)
memory usage: 44.3+ KB


In [181]:
albums.to_csv(os.path.join('data', 'AOTD.csv'), index=False)

Take a look at this :

https://www.kaggle.com/pieca111/music-artists-popularity

Hard to get it working right. Would be neat though.